# Generador de instancias
En este notebook está el código para generar los sets de instancias que se usan para experimentar.
- Estas instancias van a ser guardadas en la carpeta __instancias__.
- Cada set estará en su propia carpeta y tendrá un archivo _indice.csv_ que contendrá información sobre las instancias.

In [7]:
import random, math
import pandas as pd
from random import randrange

In [8]:
def save_instance(dataset, instance_name, n, G, H):
    with open(F"instancias/{dataset}/{instance_name}.txt", "w") as f:
        print(n, len(G),len(H),file=f)
        for aristaG in G: 
            print(str(aristaG[0])+" "+str(aristaG[1]), file=f, end="\n")
        for aristaH in H: 
            print(str(aristaH[0])+" "+str(aristaH[1]), file=f, end="\n")

def save_index(dataset, instances):
    with open(F"instancias/{dataset}/instances.txt", "w") as f:
        for instance in instances: 
            print(instance, file=f)

In [9]:
def aristas_diff(l1, l2):#devuelve las aristas de l1 que no estan en l2
    dif = [i for i in l1 if i not in l2]
    return dif
def diff(l1, l2):#devuelve la cantidad de aristas de l1 que no estan en l2
    dif = [i for i in l1 if i not in l2]
    return len(dif)

def distEucli(l1, l2):
    acum = 0
    for i  in range(0, len(l1)):
        acum = acum + (l1[i] - l2[i])** 2
    acum = acum**(1/2)
    return acum

def ordenAdy(vec,n):
    ady = []
    for i in range(int(n)+1):
        ady = ady + [[]]
    for p in vec:
        ady[p[0]] = ady[p[0]] + [p]
        ady[p[1]] = ady[p[1]] + [p]
    for i in range(int(n)+1):
        ady[i] = ady[i] + [i]
    ady.pop(0)
    ady.sort(key=len, reverse=True)
    for i in range(0,int(n)):
        ady[i] = ady[i][-1]
    return ady

def parOrd(a,b):
    if a < b:
        par = (a,b)
    else:
        par = (b,a)
    return par

def enlazar(n,caristas):
    vec = []
    for i in range(int(caristas)):
        estaEn = True
        while estaEn:
            mismos = True
            while mismos:
                a = randrange(1, int(n)+1)
                b = randrange(1, int(n)+1)
                if a != b:
                    mismos = False
            par = parOrd(a,b)
            if par not in vec:
                estaEn = False 
        vec = vec + [par]
    return vec

## Dataset 1
Instancias generadas para el peor caso en cuanto a impacto de la primer heuristica golosa constructiva que se basa en el coloreo secuencial de los vertices, la generación de las mismas está descripta en el informe.

In [10]:
filas_indice = []
for cantidad_vertices in range(10, 50):
    for caristasG in range(math.floor(cantidad_vertices/2),5*cantidad_vertices,5):
        h = []
        g = enlazar(cantidad_vertices,caristasG) #[(v,w)..]
        orden = ordenAdy(g,cantidad_vertices)#lista de los vertices ordenados por adyacencia según grado en G
        for i in range(int(int(cantidad_vertices)/2)):
            a = orden[i]
            b = orden[i + int(int(cantidad_vertices)/2)]
            par = parOrd(a,b)
            h = h + [par]
        impacto_esperado = diff(h,g)
        save_instance("peor-caso-hs1", F"HS1-AG{caristasG}-PC-{cantidad_vertices}",cantidad_vertices,g,h)
        filas_indice.append(["peor-caso-hs1", F"HS1-AG{caristasG}-PC-{cantidad_vertices}", cantidad_vertices, impacto_esperado, F"instancias/peor-caso-hs1/HS1-AG{caristasG}-PC-{cantidad_vertices}.txt"])
pd.DataFrame(filas_indice, columns=["dataset", "instancia", "cantidad_vertices", "impacto_esperado", "archivo"]).to_csv("instancias/peor-caso-hs1/indice.csv", index=False, header=True)

## Dataset 2
Instancias generadas para el mejor caso para la calidad de la primer heuristica golosa constructiva que se basa en el coloreo secuencial de los vertices, la generación de las mismas está descripta en el informe.

In [11]:
filas_indice = []
for cantidad_vertices in range(10, 50):
    for caristasG in range(math.floor(cantidad_vertices/2),5*cantidad_vertices,5):
        h = []
        g =  enlazar(cantidad_vertices,caristasG)
        orden = ordenAdy(g,cantidad_vertices)#orden de evaluacion de las aristas
        for i in range(int(cantidad_vertices)-1):
            a = orden[i]
            b = orden[i+1]
            par = parOrd(a,b)
            j = i + 2
            while par in h and j < int(cantidad_vertices):
                b = orden[j]
                par = parOrd(a,b)
                j = j+1
            if j == int(cantidad_vertices) and par in h:
                b = orden[i-1]
                par = parOrd(b,a)
                j = i - 2
                while par in h and j >= 0:
                    b = orden[j]
                    par = parOrd(a,b)
                    j = j - 1
            if par not in h:
                h = h + [par]
        impacto_esperado = diff(h,g)
        save_instance("mejor-caso-hs1", F"HS1-AG{caristasG}-MC-{cantidad_vertices}",cantidad_vertices,g,h)
        filas_indice.append(["mejor-caso-hs1", F"HS1-AG{caristasG}-MC-{cantidad_vertices}", cantidad_vertices, impacto_esperado, F"instancias/mejor-caso-hs1/HS1-AG{caristasG}-MC-{cantidad_vertices}.txt"])
pd.DataFrame(filas_indice, columns=["dataset", "instancia", "cantidad_vertices", "impacto_esperado", "archivo"]).to_csv("instancias/mejor-caso-hs1/indice.csv", index=False, header=True)

## Dataset 3
Instancias de peor caso con respecto al impacto obtenido para la segunda heuristica golosa constructiva secuencial, están descriptas en el informe en más detalle.

In [12]:
filas_indice = []
for cantidad_vertices in range(10, 50):
    for caristasH in range(math.floor(cantidad_vertices/2),5*cantidad_vertices,5):
        h = enlazar(cantidad_vertices,caristasH)
        g = []
        orden = ordenAdy(h,cantidad_vertices)
        for i in range(int(cantidad_vertices)-1):
            a = orden[i] 
            b = orden[i+1]
            par = parOrd(a,b)
            j = i + 2
            while par in g and j < int(cantidad_vertices):
                b = orden[j]
                par = parOrd(a,b)
                j = j+1
            if j == int(cantidad_vertices) and par in g:
                b = orden[i-1]
                par = parOrd(b,a)
                j = i - 2
                while par in g and j >= 0:
                    b = orden[j]
                    par = parOrd(a,b)
                    j = j - 1
            if par not in g:
                g = g + [par]
        impacto_esperado = diff(h,g) #utilizamos una cota superior para el impacto optimo
        save_instance("peor-caso-hs2", F"HS2-AH{caristasH}-PC-{cantidad_vertices}",cantidad_vertices,g,h)
        filas_indice.append(["peor-caso-hs2", F"HS2-AH{caristasH}-PC-{cantidad_vertices}", cantidad_vertices, impacto_esperado, F"instancias/peor-caso-hs2/HS2-AH{caristasH}-PC-{cantidad_vertices}.txt"])
pd.DataFrame(filas_indice, columns=["dataset", "instancia", "cantidad_vertices", "impacto_esperado", "archivo"]).to_csv("instancias/peor-caso-hs2/indice.csv", index=False, header=True)

## Dataset 4
Instancias de mejor caso de calidad para la segunda heuristica constructiva golosa, están descriptas en el informe en más detalle.

In [13]:
filas_indice = []
for cantidad_vertices in range(10, 50):
    for caristasH in range(math.floor(cantidad_vertices/2),5*cantidad_vertices,5):
        h = enlazar(cantidad_vertices, caristasH)
        g = []
        orden = ordenAdy(h,cantidad_vertices)
        for i in range(int(int(cantidad_vertices)/2)):
            a = orden[i]
            b = orden[i + int(int(cantidad_vertices)/2)]
            par = parOrd(a,b)
            g = g + [par]
        impacto_esperado = diff(h,g) #utilizamos una cota superior para el impacto optimo
        save_instance("mejor-caso-hs2", F"HS2-AH{caristasH}-MC-{cantidad_vertices}",cantidad_vertices,g,h)
        filas_indice.append(["mejor-caso-hs2", F"HS2-AH{caristasH}-MC-{cantidad_vertices}", cantidad_vertices, impacto_esperado, F"instancias/mejor-caso-hs2/HS2-AH{caristasH}-MC-{cantidad_vertices}.txt"])
pd.DataFrame(filas_indice, columns=["dataset", "instancia", "cantidad_vertices", "impacto_esperado", "archivo"]).to_csv("instancias/mejor-caso-hs2/indice.csv", index=False, header=True)

# Dataset 6
Instancias generadas de forma aleatoria para una gran aplitud de valores.

In [14]:
filas_indice = []
for cantidad_vertices in range(10, 111,2):
    for caristasH in range(math.floor(cantidad_vertices/2),5*cantidad_vertices,10):
        for caristasG in range(math.floor(cantidad_vertices/2),5*cantidad_vertices,10):
            h = enlazar(cantidad_vertices,caristasH)
            g = enlazar(cantidad_vertices,caristasG)
    
            impacto_esperado = diff(h,g) #utilizamos una cota superior para el impacto optimo
            save_instance("instancias-costo", F"IC-AG{caristasG}-AH{caristasH}-n{cantidad_vertices}",cantidad_vertices,g,h)
            filas_indice.append(["instancias-costo", F"IC-AG{caristasG}-AH{caristasH}-n{cantidad_vertices}", cantidad_vertices, impacto_esperado, F"instancias/instancias-costo/IC-AG{caristasG}-AH{caristasH}-n{cantidad_vertices}.txt"])
pd.DataFrame(filas_indice, columns=["dataset", "instancia", "cantidad_vertices", "impacto_esperado", "archivo"]).to_csv("instancias/instancias-costo/indice.csv", index=False, header=True)

# Dataset 7
Instancias provistas por la catedra para la medición de calidad de nuestro algoritmos.

In [15]:
#solamente generamos el indice, dado que las instancias ya nos fueron provistas por la catedra
filas_indice = []
filas_indice.append(["instancias-calidad", F"CMI_n6", 6, 1, F"instancias/instancias-calidad/CMI_n6.in"])
filas_indice.append(["instancias-calidad", F"CMI_n8", 8, 6, F"instancias/instancias-calidad/CMI_n8.in"])
filas_indice.append(["instancias-calidad", F"CMI_n10", 10, 3, F"instancias/instancias-calidad/CMI_n10.in"])
filas_indice.append(["instancias-calidad", F"CMI_n12", 12, 16, F"instancias/instancias-calidad/CMI_n12.in"])
filas_indice.append(["instancias-calidad", F"CMI_n14", 14, 12, F"instancias/instancias-calidad/CMI_n14.in"])
filas_indice.append(["instancias-calidad", F"CMI_n16", 16, 20, F"instancias/instancias-calidad/CMI_n16.in"])
filas_indice.append(["instancias-calidad", F"CMI_n18", 18, 27, F"instancias/instancias-calidad/CMI_n18.in"])
filas_indice.append(["instancias-calidad", F"CMI_n20", 20, 25, F"instancias/instancias-calidad/CMI_n20.in"])
filas_indice.append(["instancias-calidad", F"CMI_n22", 22, 26, F"instancias/instancias-calidad/CMI_n22.in"])
filas_indice.append(["instancias-calidad", F"CMI_n24", 24, 33, F"instancias/instancias-calidad/CMI_n24.in"])
filas_indice.append(["instancias-calidad", F"CMI_n26", 26, 38, F"instancias/instancias-calidad/CMI_n26.in"])
filas_indice.append(["instancias-calidad", F"CMI_n28", 28, 48, F"instancias/instancias-calidad/CMI_n28.in"])
filas_indice.append(["instancias-calidad", F"CMI_n30", 30, 47, F"instancias/instancias-calidad/CMI_n30.in"])
pd.DataFrame(filas_indice, columns=["dataset", "instancia", "cantidad_vertices", "impacto_esperado", "archivo"]).to_csv("instancias/instancias-calidad/indice.csv", index=False, header=True)